In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import nltk
import zipfile
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# ============================================================
# 0. NLTK SETUP
# ============================================================
nltk.download("stopwords")
nltk.download("wordnet")

# ============================================================
# 1. LOAD INPUT FILE (SAFE LOADER)
# ============================================================

file_path = "/home/sagemaker-user/CSAT/data/OCT_NEW_TRANSCRIPTS.xlsx"

def safe_read_input(file_path):
    """Reads file safely even if XLSX is corrupted."""

    # Case 1 — Check if XLSX is valid ZIP
    if zipfile.is_zipfile(file_path):
        print("✔ Valid Excel file detected. Reading with openpyxl…")
        return pd.read_excel(file_path, engine="openpyxl", dtype=str)

    # Case 2 — Not a ZIP → treat as CSV
    print("⚠ File is NOT a valid Excel. Trying CSV parsing…")
    return pd.read_csv(file_path, dtype=str, encoding="utf-8", on_bad_lines="skip")

df = safe_read_input(file_path)
df = df.copy()

# Force ALL columns to string to prevent truncation
df = df.astype(str)

# If CX1 ID exists → ensure no truncation
for col in df.columns:
    if "cx1" in col.lower() or "call" in col.lower() or "id" in col.lower():
        df[col] = df[col].astype(str)


# ============================================================
# 2. REMOVE EMPTY TRANSCRIPTS
# ============================================================
NO_TEXT = ["", "nan", "none", "n/a", "[]"]

summary_txt = df["Summary"].astype(str).str.strip().str.lower()
df = df[~summary_txt.isin(NO_TEXT)].copy()

# ============================================================
# 3. TIMESTAMP CLEANING
# ============================================================
def parse_ts(x):
    try:
        if isinstance(x, (int, float)) and float(x) > 30000:
            return pd.to_datetime("1899-12-30") + pd.to_timedelta(float(x), unit="D")
        return pd.to_datetime(x, errors="coerce")
    except:
        return pd.NaT

if "timestamp" in df.columns:
    df["timestamp"] = df["timestamp"].apply(parse_ts)
    df["timestamp"] = df["timestamp"].fillna(method="ffill")

    df["Date"] = df["timestamp"].dt.normalize()
    df["YearMonth"] = df["Date"].dt.to_period("M").astype(str)
    df["Week"] = df["Date"].dt.to_period("W").astype(str)

    df["timestamp_str"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
    df["Date_str"] = df["Date"].dt.strftime("%Y-%m-%d")

# ============================================================
# 4. COMPLAINT DICT EXPANSION
# ============================================================
def parse_dict(val):
    if pd.isna(val) or val == "":
        return {}
    try:
        fixed = (
            str(val)
            .replace("true", "True")
            .replace("false", "False")
            .replace("null", "None")
        )
        return ast.literal_eval(fixed)
    except:
        return {}

if "Complaint" in df.columns:
    df["Complaint_dict"] = df["Complaint"].apply(parse_dict)
    complaints = pd.json_normalize(df["Complaint_dict"]).add_prefix("Complaint_")
else:
    complaints = pd.DataFrame(index=df.index)

# ============================================================
# 5. FLAGS EXPANSION
# ============================================================
if "flags_list" in df.columns:
    df["flags_list"] = df["flags_list"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df["flags_list"] = df["flags_list"].apply(lambda x: x if isinstance(x, list) else [])

    max_flags = df["flags_list"].apply(len).max()

    flag_frames = []

    for i in range(max_flags):
        tmp = df["flags_list"].apply(lambda lst: lst[i] if i < len(lst) else {})
        tmp_df = pd.json_normalize(tmp).add_prefix(f"Flag{i+1}_")
        flag_frames.append(tmp_df)

    flags_expanded = pd.concat(flag_frames, axis=1) if max_flags > 0 else pd.DataFrame(index=df.index)
else:
    flags_expanded = pd.DataFrame(index=df.index)

# ============================================================
# 6. NLP CLEANING
# ============================================================
stop_words = set(stopwords.words("english"))
extra_words = {
    "please","kindly","hello","hi","thank","thanks","sir","madam","customer",
    "call","agent","support","yeah","ok","okay"
}
stop_words.update(extra_words)

lemmatizer = WordNetLemmatizer()

def clean_summary(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

df["Summary_Clean"] = df["Summary"].astype(str).apply(clean_summary)

# ============================================================
# 7. MERGE ALL FINAL COMPONENTS
# ============================================================
df_final = pd.concat(
    [
        df.drop(columns=["Complaint_dict", "flags_list"], errors="ignore"),
        complaints,
        flags_expanded
    ],
    axis=1
)

# ============================================================
# 8. SAVE FINAL OUTPUT WITHOUT TRUNCATION (STRING SAFE)
# ============================================================

output_path = "FINAL_TRANSCRIPTS_POWERBI_SAFE.xlsx"

# force all IDs to text
df_final = df_final.astype(str)

df_final.to_excel(output_path, index=False, engine="openpyxl")

print("🎉 FINAL FILE CREATED SUCCESSFULLY →", output_path)
